# 6. Training a Neural Network using TensorFlow

In [81]:
import os.path, pandas as pd 
import matplotlib as mp, matplotlib.pyplot as plt
import numpy as np, json, random, ast
from IPython.display import clear_output
from pandas.io.json import json_normalize, read_json
from datetime import datetime
from IPython.display import HTML
import cesiumpy

import tensorflow as tf
import tensorboard as tb
import sys

import sklearn as skl
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report

#Set some parameters for nicer visualizations
pd.set_option('display.expand_frame_repr', False) #do not wrap the printout of Pandas DataFrames
pd.set_option('display.precision', 2)
pd.set_option('display.max_columns', 50)
mp.rcParams['figure.figsize'] = (15, 15)
mp.pyplot.style.use = 'fivethirtyeight'
%matplotlib inline

In [108]:
#get data, normalize it, check dimensions
dfmain = pd.read_csv('data/dffinal.csv', index_col=0)
dfmain['geojson'] = dfmain['geojson'].map(ast.literal_eval) #convert string to dict
dfmain['sta'] = pd.to_datetime(dfmain['sta'])
dfmain['end'] = pd.to_datetime(dfmain['end'])
dfmain['duration'] = pd.to_timedelta(dfmain['duration'])   
dfmain.sample()

X = np.array(dfmain[['humidity', 'dewPoint', 'temperature', 
                    'pressure', 'visibility', 'windSpeed', 
                    'precipIntensity', 'precipProbability', 
                    'dat.cape', 'crosswind', 'ratio']])
y = np.array(dfmain['regulated'], dtype='float')

y.astype('float')

X_train, X_test, y_train, y_test = skl.model_selection.train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)


# Apply SMOTE on the training set:
from imblearn.over_sampling import SMOTE, ADASYN
smo = SMOTE(ratio='all', kind='regular', random_state=42)
X_train, y_train = smo.fit_sample(X_train, y_train)

adas = ADASYN(ratio='minority', random_state=42)
X_train, y_train = adas.fit_sample(X_train, y_train)


y.shape = (y.size, 1)
y_train.shape = (y_train.size, 1)
y_test.shape = (y_test.size, 1)

print(X_train.shape, y_train.shape)

(7110, 11) (7110, 1)


In [92]:
# Standardization
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [93]:
# concatenate y vectors from [0] to [0 1]
y_train = np.concatenate([1 - y_train, y_train], axis =1 )
y_test = np.concatenate([1 - y_test, y_test], axis =1 )

### Tensorflow Graph Construction

In [102]:
'''
adapted from
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

# Parameters
learning_rate = 0.003
training_epochs = 110
batch_size = 400
display_step = 10

# Network Parameters
n_hidden_1 = 15 # 1st layer number of features
n_hidden_2 = 15 # 2nd layer number of features
n_hidden_3 = 15
n_input = 11 # number of features
n_classes = 2 # number of class variables (1 means one variable with possible values 0 and 1)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# model factory
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.tanh(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Hidden layer with RELU activation
    layer_3 = tf.add(tf.matmul(layer_1, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)    
    # Output layer with linear activation
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss, optimizer and AUC
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
_, aucscore = tf.metrics.auc(labels=tf.argmax(y, 1), predictions=tf.argmax(pred, 1) )

# UPDATE: modify cost function such that the imbalanced dataset is respected
class_weights = tf.constant([[1.0, 12.0]])
costweights =  tf.reduce_sum(class_weights * y, axis = 1)
weighted_cost = cost * costweights
cost = weighted_cost    


### Training

In [103]:
# Initializing the variables
init = tf.global_variables_initializer()
#init2 = tf.local_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    sess.run(tf.initialize_all_variables())
    sess.run(tf.initialize_local_variables())
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(X_train.size / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            start = i * batch_size
            end =  i * batch_size + batch_size
            X_train_batch = X_train[start:end, :]
            y_train_batch = y_train[start:end, :]            
            
            # Run optimization op (backprop) and cost op (to get loss value)
            result = sess.run([optimizer, cost], feed_dict={x: X_train_batch, y: y_train_batch})
            # Compute average aucscore on the batch
            # sess.run(tf.initialize_local_variables())
            aucs = sess.run(aucscore, feed_dict={x : X_train_batch, y : y_train_batch})
            
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "aucscore=", "{:.9f}".format(aucs))
            sys.stdout.flush()
    print("Optimization Finished!")

    # Evaluation:
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: y_test}))
    
    # print final AUC
    sess.run(tf.initialize_all_variables())
    sess.run(tf.initialize_local_variables())
    print(sess.run(aucscore, feed_dict={x : X_test, y : y_test} ))

Epoch: 0001 aucscore= 0.500000000
Epoch: 0011 aucscore= 0.734552205
Epoch: 0021 aucscore= 0.802620471
Epoch: 0031 aucscore= 0.833911777
Epoch: 0041 aucscore= 0.853651404
Epoch: 0051 aucscore= 0.867936492
Epoch: 0061 aucscore= 0.879143000
Epoch: 0071 aucscore= 0.887951374
Epoch: 0081 aucscore= 0.895048678
Epoch: 0091 aucscore= 0.900943100
Epoch: 0101 aucscore= 0.906152189
Optimization Finished!
Accuracy: 0.880159
0.462785
